In [69]:
import pandas as pd
import numpy as np

In [70]:
import os

book1_path = "../data/law-mds/Book I/"
book1 = []

book2_path = "../data/law-mds/Book II/"
book2 = []


if os.path.exists(book1_path):
    book1_files = os.listdir(book1_path)
    for file in book1_files:
        with open(book1_path + file, "r") as f:
            book1.append(f.read())
else:
    print("Directory not found.")

if os.path.exists(book1_path):
    book2_files = os.listdir(book2_path)
    for file in book2_files:
        with open(book2_path + file, "r") as f:
            book2.append(f.read())
else:
    print("Directory not found.")

In [71]:
def parse_part(md: str):
    # lines = [line+"\n" for line in md.split("\n")]
    lines = md.split("\n")
    print(lines)

    result = {}

    current_part = None
    current_part_title = None
    current_chapter = None
    current_chapter_title = None
    current_section = None
    current_section_title = None

    for line in lines:
        if line == '':
            continue

        if line.startswith("#") and not line.startswith("##"):
            actual = line.strip("#").strip().upper()
            start = actual.split(":")[0].strip()

            if start.startswith("PART"):
                current_part = start
                current_part_title = actual.split(":")[1].strip()
                result[current_part] = {'title': current_part_title, 'chapters': {}}
                continue

            if start.startswith("CHAPTER"):
                current_chapter = start
                current_chapter_title = actual.split(":")[1].strip()
                result[current_part]['chapters'][current_chapter] = {'title': current_chapter_title, 'sections': {}}
                continue

            else: 
                print("UNKNOWN START", start)
                continue

        elif line.startswith("#"):
            actual = line.strip("#").strip().upper()
            start = actual.split(":")[0].strip()
            
            if start.startswith("CHAPTER"):
                current_chapter = start
                current_chapter_title = actual.split(":")[1].strip()
                result[current_part]['chapters'][current_chapter] = {'title': current_chapter_title, 'sections': {}}
                continue

            elif start.startswith("SECTION"):
                current_section = start
                # print(current_section)
                current_section_title = actual.split(":")[1].strip()
                result[current_part]['chapters'][current_chapter]['sections'][current_section] = {'title': current_section_title, 'content': ""}
                continue

            else:
                print("UNKNOWN START", start)
                continue
        else:
            if current_part and current_chapter and current_section:
                result[current_part]['chapters'][current_chapter]['sections'][current_section]['content'] += (line+"\n")

    return result


In [72]:
law = {
    "Book I": parse_part("\n".join(book1)),
    "Book II": parse_part("\n".join(book2))
}

['# Part I: CRIMINAL LAW APPLICATION OF CRIMINAL LAW', '', '# Chapter 1: PRELIMINARY PROVISIONS', '', '## SECTION 1: Content', '', '1. Book I comprising Sections 1 to 101;', '2. Book II comprising Sections 102 to 361;', '3. The decree relating to regulatory provisions of the Penal Code which define offences, comprising Sections 362 to 370;', '4. Sections 371 and 372, which lay down the transitional and final', 'provisions.', '', '### SECTION 1-1: No exemption', '', 'All persons shall be subject to the criminal law.', '', '## SECTION 2:\xa0General and special application', '', '1. This Code and every provision of criminal law shall be subject to the rules of international law and to ail treaties duly promulgated and published. ', '2. Book I of this Code shall govern ail other criminal law, unless special provision has been made, even before the entry into force of this book, concerning particularly the exclusion of suspension and the exclusion or limitation of mitigating circumstances b

In [73]:
import json

with open("../data/criminal_law.json", "w") as f:
    json.dump(law, f)